#  Jogo dos Rastros
## Projeto nº 2
### Introdução à Inteligência Artificial edição 2020/21


## Grupo: 42

### Elementos do Grupo

Nome: Ivo Veiga

Número: 44865

Nome: João Silva

Número: 48782

Nome: Manuel Tovar

Número: 49522

### Explicação da função e sua lógica
(descrição)

In [18]:
from rastros import *

#win_or_loss = state.compute_utility(player)    
#if win_or_loss != 0:
#    return win_or_loss * 5

def fun_aval_42(state, player):
    goal = (8, 1)
    no_goal = (1, 8)
    if player == "N":
        goal = (1, 8)
        no_goal = (8, 1)
    
    if state.white == goal:
        return 5
    elif state.white == no_goal:
        return -5
    elif len(state.moves()) == 0:
        return -2 if state.to_move == player else 2
    else:        
        score_white_goal = 1.0 / (2 ** distancia(state.white, goal))
        sum_score_moves = 0
        for move in state.moves():
            score_move_goal = 1.0 / (2 ** distancia(move, goal))
            sum_score_moves += score_move_goal
        
        return (sum_score_moves * 1) + (score_white_goal * 1.0)
    

#########################################################################################################
### APAGAR NO FIM #######################################################################################
def pergunta_print(game, state):
    state.display()
    print("ultima pos: ", list(state.blacks)[-1])
    if len(state.blacks) <= 1:
        pergunta_print.tst = 0
    pergunta_print.tst = pergunta_print.tst + 1
    print(pergunta_print.tst)
    print("Jogadas possíveis: ", state.moves())
    return eval(input(state.to_move+", para onde quer jogar? "))
    
humano_print = Jogador("print1", pergunta_print)
basilio_test = Jogador("Basilio_test", lambda game, state:
                  alphabeta_cutoff_search_new(state,game,depth_for_all,eval_fn=f_aval_basico))
#########################################################################################################
#########################################################################################################

### Demonstração
(Mostrem que o código está a funcionar, com exemplos adequados, executem um ou mais jogos, mostrem os seus resultados, exibam situações de jogo que demonstrem que a lógica da vossa função está correctamente implementada.)

In [20]:
jogador42 = Jogador("jogador42",
                  lambda game, state:
                  alphabeta_cutoff_search_new(state,game,depth_for_all,eval_fn=fun_aval_42))

#jogo = jogaRastros11(basilio,jogador42)
#print(jogo)

def mostraJogaRastrosNN(n_c, listaJog, listaAdv, nsec=1, verbose=False):
    lista_jogos=[]
    for jog in listaJog:
        for adv in listaAdv:
            if jog != adv:
                lista_jogos.append((jog.nome, adv.nome, jogaRastros11com_timeout(jog,adv, nsec)))
                if verbose:
                    print("\nZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ")
                    print("JOGO " + str(n_c + 1) + " com " + jog.nome + "=S e " + adv.nome + "=N")
                    mostraJogo(lista_jogos[-1][2][0], verbose = True)
                    print("ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ")                
    return lista_jogos      

def n_faz_campeonato(num_campeonatos, listaJogadores, nsec=10, verbose=False):
    jogador_vitorias = {}
    print("Progresso com " + str(num_campeonatos) + " campeonatos: ", end='')
    for n_c in range(num_campeonatos):
        campeonato = mostraJogaRastrosNN(n_c, listaJogadores, listaJogadores, nsec, verbose)        
        resultado_jogos = [(a,b,n) for (a,b,(x,n)) in campeonato]
        tabela = dict([(jog.nome, 0) for jog in listaJogadores])
        for jogo in resultado_jogos:
            if jogo[2] == 1:
                tabela[jogo[0]] += 1
            else:
                tabela[jogo[1]] += 1
        classificacao = list(tabela.items())
        classificacao.sort(key=lambda p: -p[1])
        for jog in classificacao:
            jogador_vitorias[jog[0]] = jogador_vitorias.get(jog[0],0) + jog[1]
        print(str(n_c + 1) + " ", end='')
    print("\nJOGADOR", "VITÓRIAS")
    for jogador in jogador_vitorias:
        print('{:11}'.format(jogador), '{:>4}'.format(jogador_vitorias[jogador]))

n_faz_campeonato(10, [jogador42, basilio], verbose = True)

Progresso com 10 campeonatos: 1 2 3 4 5 6 7 8 9 10 
JOGADOR VITÓRIAS
jogador42      9
Basilio       11
